<a href="https://colab.research.google.com/github/Dan22dongwon/Pandas-sci-kit-learn/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#자연어 처리
- NLP(Natural Language Procewssing)
- 사람이 사용하는 언어를 처리하고 해석


# 자연어 처리 기초

# 형채소 분석기
- Konlpy : 한국어 처리를 위한 형태소 분석기
- 형태소 : 의미를 가지는 요소로서는 더 이상 분석할 수 없는 가장 작은 말의 단위

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# konlpy 설치
!pip install konlpy

In [ ]:
# 라이브러리 불러오기 (okt)  okt 이외에도 한국어 형태소 관련 라이브러리가 있음
import konlpy
from konlpy.tag import Okt
tokenizer = Okt()

In [ ]:
# 토큰화 (형태소 단위)
text = "함께 탐험하며 성장하는 AI학교 AIFFEL"
tokenizer.morphs(text)

In [ ]:
# 토큰화 (명사만 추출)
tokenizer.nouns(text)

In [ ]:
# 토큰화 (품사, 태깅)
tokenizer.pos(text)

# CountVextorizer

In [ ]:
# CountVextorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [ ]:
# 단어 토큰화 (Okt)
words = tokenizer.morphs(text)

In [ ]:
# 데이터 학습
vect.fit(words)

In [ ]:
# 학습된 어휘
vect.get_feature_names_out()

In [ ]:
# 단어 사전
vect.vocabulary_

In [ ]:
# 단어 사전 크기
len(vect.vocabulary_)

In [ ]:
# 인코딩
df_t = vect.transform(words)

In [ ]:
# 인코딩 데이커 Matrix
df_t.toarray()

In [ ]:
#어휘와 피처 (데이터 프레임)
pd.DataFrame(df_t.toarray(), columns=vect.get_feature_names_out())

In [ ]:
# test
test = "AI 공부하며 함께 성장해요!"

In [ ]:
# 단어 토큰화 (Okt)
words = tokenizer.morphs(test)
words

In [ ]:
# 인코딩 데이터 Matrix
test_t = vect.transform(words)
test_t.toarray()

In [ ]:
# 어휘와 피처 (데이터 프레임)
pd.DataFrame(df_t.toarray(), columns=vect.get_feature_names_out())

# TfidfVextorizer

In [ ]:
# tf_idf
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#tf_idf 활용 어휘 사전 구축
vect = TfidfVectorizer()
words = tokenizer.morphs(text)
vect.fit(words)
vect.vocabulary_

In [ ]:
# 인코딩된 데이터 Matrix
vect.transform(words).toarray()

# 감성 분석
 
 문제정의 -> 탐색적 데이터 분석EDA -> 데이터 전처리 (토큰화,어휘 사전 구축, 인코딩) -> 모델학습 -> 예측(긍정,부정)



- 데이터 불러오기

In [ ]:
# 라이브러리 불러오기
import pandas as pd

In [ ]:
# 데이터 불러오기
url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
df = pd.read_csv(url, sep="\t")

- EDA 및 데이터 전처리

In [ ]:
# 데이터 샘플
df.head()

In [ ]:
#데이터 크기
df.shape

In [ ]:
#타겟 확인
df['label'].value_counts()

In [ ]:
# 결측치
df.isnull().sum()

In [ ]:
# 결측치 ㅐ삭제
print(df.shape)
df = df.dropna()
print(df.shape)

In [ ]:
# 피처 엔지니어링(문장의 길이)
df['len'] = df['document'].apply(len)
df.head()

In [ ]:
# len 시각화 (label == 0)
import matplotlib.pyplot as plt
df[df.label==0]['len'].plot(kind='hist')

In [ ]:
# len 시각화 (label == 1)
import matplotlib.pyplot as plt
df[df.label==1]['len'].plot(kind='hist')

In [ ]:
# 데이터 샘플링 df[: 1000]
df = df[:1000]
df.shape

In [ ]:
# 토큰화
vect = CountVectorizer(tokenizer=tokenizer.morphs)
vectors = vect.fit_transform(df['document'])

- 머신러닝

In [ ]:
# 머신러닝 -> 교차검증 (f1)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(random_state=2022)
cross_val_score(model, vectors, df['label'], scoring='f1', cv=5).mean()

# 자연어 전처리
- 어휘사전 구축
  - 00% 이상 나타나는 단어 무시
  - 최소 N개의 문장에만 나타나는 단어만 유지

- 불용어(stopword)
  - 큰 의미가 없는 단어
  - 예) 은,는,이,가,을.여기,저기

- 띄어쓰기
_ 반복되는 글자 정리
- 맞춤법 검사

1. 어휘 사전 구축

In [ ]:
# 토큰화(max_df) N개 보다 큰 단어 수 무시
vect = CountVectorizer(tokenizer=tokenizer.morphs, max_df=10)
vectors = vect.fit_transform(df['document'])
model = RandomForestClassifier(random_state=2022)
cross_val_score(model, vectors, df['label'], scoring='f1', cv=5).mean()

In [ ]:
# 토큰화 (min_df) N개 보다 작은 단어 수 무시 
vect = CountVectorizer(tokenizer=tokenizer.morphs, min_df=1)
vectors = vect.fit_transform(df['document'])
model = RandomForestClassifier(random_state=2022)
cross_val_score(model, vectors, df['label'], scoring='f1', cv=5).mean()

 2. 불용어(stopword)

In [ ]:
# stop_words
text = "함께 탐험하며 성장하는 AI 학교 AIFFEL"
stop_words = ['함께','ai']
vect = CountVectorizer(stop_words = stop_words)
words = tokenizer.morphs(text)
vect.fit(words)
vect.vocabulary_

3. 띄어쓰기
  - 더 자세히 : https://github.com/haven-jeon/PykoSpacing
  https://github.com/jungin500/py-hanspell

  오류 관련 정보 : https://github.com/ssut/py-hanspell/issues/31

In [ ]:
# Spacing 설치
!pip install git+https://github.com/jungin500/py-hanspell

In [ ]:
# 띄어쓰기

from hanspell import spell_checker

text = "함께탐험하며성장하는AI학교AIFFEL" 
result = spell_checker.check(text)
corrected_text = result.checked

print(corrected_text)

3. 반복되는 글자 정리
 - 더 자세히 : https://github.com/lovit/soynlp

In [ ]:
# soynlp 설치
!pip install soynlp

In [ ]:
# 댓글 데이터에 등장하는 반복되는 이모티콘의 정리
from soynlp.normalizer import *
emoticon_normalize('하하하하하ㅋㅋㅋㅋㅋ호호호호호ㅠㅠㅠㅠㅜㅜㅜㅜㅜ', num_repeats=3)

4. 맞춤법 검사기
  - 더 자세히 : https://github.com/ssut/py-hanspell

In [ ]:
# py-hanspell 설치
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
# 맞춤법 검사
from hanspell import spell_checker
text = '알파고 이전, 2015년부터 만들 어진 최초의AI 커뮤니티 모두의연구소,학연,지연,모두연이라는 말이나올마큼 AI의 보금자리로서 중요한 역할을 하고있다'

In [ ]:
# 수정된 문장
result = spell_checker.check(text)

In [ ]:
result.checked